In [1]:
import findspark
findspark.init('/home/lss9/anaconda3/lib/python3.7/site-packages/pyspark/')

In [2]:
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
import re
from tqdm import tqdm

In [3]:
sc = SparkContext()

In [4]:
sqlContext = SQLContext(sc)

In [5]:
data = sqlContext.read.json('/home/luciano/google/tables.json')

In [6]:
data.printSchema()

root
 |-- _id: string (nullable = true)
 |-- numCols: long (nullable = true)
 |-- numDataRows: long (nullable = true)
 |-- numHeaderRows: long (nullable = true)
 |-- numericColumns: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- order: double (nullable = true)
 |-- pgId: long (nullable = true)
 |-- pgTitle: string (nullable = true)
 |-- sectionTitle: string (nullable = true)
 |-- tableCaption: string (nullable = true)
 |-- tableData: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cellID: long (nullable = true)
 |    |    |    |-- isNumeric: boolean (nullable = true)
 |    |    |    |-- subtableID: long (nullable = true)
 |    |    |    |-- surfaceLinks: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- endOffset: long (nullable = true)
 |    |    |    |    |    |-- inTemplate: boolean (nullable = 

In [7]:
data.registerTempTable("tables")

This function removes the special characters from a string

In [8]:
def clearString(text):
    
    text = re.sub('[^A-Za-z]+',' ',text)
    
    text = text.lstrip()
    
    text = text.rstrip()
    
    return text

getting the page ID, page title, section title and table caption from raw data

In [ ]:
returnSelect = sqlContext.sql("select pgId, pgTitle, sectionTitle, tableCaption from tables").collect()

In [ ]:
formattedTables = []

for row in tqdm(returnSelect):
    
    dataPgID = row.pgId
    dataPgTitle = row.pgTitle
    dataSectionTitle = row.sectionTitle
    dataTableCaption = str(row.tableCaption)
    
    #removing special characters
    dataPgTitle = clearString(dataPgTitle)
    dataSectionTitle = clearString(dataSectionTitle)
    dataTableCaption = clearString(dataTableCaption)
    
    formattedTables.append([dataPgID,dataPgTitle,dataSectionTitle,dataTableCaption])

100%|██████████| 1652771/1652771 [00:16<00:00, 100806.40it/s]


In [ ]:
returnSelect = sqlContext.sql("select tableData from tables").collect()

In [ ]:
tableText = []

for tablesRawData in tqdm(returnSelect):
    
    tableBody = ''
    
    tableRawData = tablesRawData.tableData
    
    for tableLines in tableRawData:
            
        tableRawColumns = tableLines
        
        for tableColumns in tableRawColumns:
            
            tableBody = tableBody +' '+ tableColumns.text
            
    tableBody = clearString(tableBody)
    
    tableText.append(tableBody)

In [ ]:
len(tableText)

In [10]:
returnSelect = sqlContext.sql("select tableHeaders from tables where pgID = 4194312").collect()

In [12]:
tableHeaderData = []

for tablesHeaderRawData in tqdm(returnSelect):
    
    tableHeader = ''
    
    tablesHeaderRawData = tablesHeaderRawData.tableHeaders
    
    for tableHeaderLines in tablesHeaderRawData:
            
        tableHeaderRawColumns = tableHeaderLines
        
        for tableHeaderColumns in tableHeaderRawColumns:
            
            tableHeader = tableHeader +' '+ tableHeaderColumns.text
            
    tableHeader = clearString(tableHeader)
    
    tableHeaderData.append(tableHeader)

100%|██████████| 3/3 [00:00<00:00, 13273.11it/s]


In [13]:
tableHeaderData

['Professional ratings Professional ratings Review scores Review scores Source Rating',
 'Chart Peak position',
 'Region Date Label']